In [1]:
!pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ------- -------------------------------- 2.1/10.6 MB 11.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/10.6 MB 11.7 MB/s eta 0:00:01
   ------------------------ --------------- 6.6/10.6 MB 11.2 MB/s eta 0:00:01
   --------------------------------- ------ 8.9/10.6 MB 11.3 MB/s eta 0:00:01
   -------------------------------------- - 10.2/10.6 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 9.7 MB/s eta 0:00:00


In [ ]:
import pygame
import math
import random

# Initialize pygame
pygame.init()

# Screen setup
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Robot Navigation")

# Constants
ROBOT_RADIUS = 15
OBSTACLE_RADIUS = 20
SENSOR_RANGE = 40
LINEAR_VELOCITY = 1
ANGULAR_VELOCITY = 3

# Colors
WHITE, BLACK, RED, BLUE = (255, 255, 255), (0, 0, 0), (255, 0, 0), (0, 0, 255)

# Robot position and goal
robot_pos, robot_angle = [100, 100], 0
goal_pos = [700, 500]

# Generate obstacles
obstacles = [(random.randint(50, WIDTH - 50), random.randint(50, HEIGHT - 50)) for _ in range(10)]

def distance(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

def detect_obstacle(robot_pos):
    """I will detect obstacles within the sensor range."""
    return next(((obs, obs) for obs in obstacles if distance(robot_pos, obs) <= SENSOR_RANGE + OBSTACLE_RADIUS), (None, None))

def rotate(angle, direction):
    """Iam going to rotate the robot to the left or right."""
    return (angle + (ANGULAR_VELOCITY if direction == "left" else -ANGULAR_VELOCITY)) % 360

def move_towards_goal():
    """we have to move robot towards the goal."""
    global robot_angle
    angle_to_goal = math.degrees(math.atan2(goal_pos[1] - robot_pos[1], goal_pos[0] - robot_pos[0]))
    if abs(robot_angle - angle_to_goal) > 5:
        robot_angle = rotate(robot_angle, "left" if angle_to_goal > robot_angle else "right")
    else:
        robot_pos[0] += LINEAR_VELOCITY * math.cos(math.radians(robot_angle))
        robot_pos[1] += LINEAR_VELOCITY * math.sin(math.radians(robot_angle))

def avoid_obstacle(obstacle_pos):
    """we have to avoid the obstacle by deciding to turn left or right based on its position."""
    global robot_angle
    robot_pos[0] -= 10 * math.cos(math.radians(robot_angle))  # Back off slightly

    # Calculate angle to the obstacle
    angle_to_obstacle = math.degrees(math.atan2(obstacle_pos[1] - robot_pos[1], obstacle_pos[0] - robot_pos[0]))
    angle_diff = (angle_to_obstacle - robot_angle) % 360

    # Decision-making for turning left or right
    if 0 < angle_diff < 180:  # Obstacle is in front
        robot_angle = rotate(robot_angle, "right")  # Turn right
    else:  # Obstacle is on the left side or behind
        robot_angle = rotate(robot_angle, "left")  # Turn left

    # Move forward after turning
    robot_pos[0] += LINEAR_VELOCITY * math.cos(math.radians(robot_angle))
    robot_pos[1] += LINEAR_VELOCITY * math.sin(math.radians(robot_angle))

running = True
while running:
    screen.fill(WHITE)
    pygame.draw.circle(screen, BLUE, goal_pos, 10)
    for obs in obstacles:
        pygame.draw.circle(screen, RED, obs, OBSTACLE_RADIUS)
    pygame.draw.circle(screen, BLACK, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)

    obstacle_detected, obstacle_pos = detect_obstacle(robot_pos)
    if obstacle_detected:
        avoid_obstacle(obstacle_pos)
    else:
        move_towards_goal()

    if distance(robot_pos, goal_pos) < 10:
        print("Goal reached!")
        running = False

    pygame.display.flip()
    pygame.time.delay(50)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()
